In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_page_content(request_url):
    #得到页面的内容
    headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64;) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.146 Safari/537.36'}
    html = requests.get(request_url, headers = headers, timeout = 10)
    content = html.text
    #通过Content创建BeautifulSoup对象
    soup = BeautifulSoup(content,'html.parser', from_encoding = 'utf-8')
    return soup 

#分析当前投诉页面信息
def analysis(soup):
    df = pd.DataFrame(columns = ['id', 'brand', 'car_model', 'type', 'desc', 'problem', 'datetime',\
                                'status'])
    #找到完整的投诉信息框
    temp = soup.find('div', class_ = 'tslb_b')
    #找出所有的tr,即行
    tr_list = temp.find_all('tr')
    for tr in tr_list:
        td_list = tr.find_all('td')
        #如果没有td,就说明是表头
        if len(td_list) > 0:
            #投诉编号 投诉品牌 投诉车系 投诉车型 问题简述 典型问题 投诉时间 投诉状态
            id, brand, car_model, type, desc, problem, datetime, status = \
                td_list[0].text, td_list[1].text, td_list[2].text, td_list[3].text, \
                td_list[4].text, td_list[5].text, td_list[6].text, td_list[7].text, 
            temp = {}
            temp['id'] = id
            temp['brand'] = brand
            temp['car_model'] = car_model
            temp['type'] = type
            temp['desc'] = desc
            temp['problem'] = problem
            temp['datetime'] = datetime
            temp['status'] = status
            df = df.append(temp, ignore_index = True)
    return df

result = pd.DataFrame(columns = ['id', 'brand', 'car_model', 'type', 'desc', 'problem', 'datetime',\
                                'status'])
#请求url
base_url = 'http://www.12365auto.com/zlts/0-0-0-0-0-0_0-0-0-0-0-0-0-' 
page_number = 10
for i in range(page_number):
    #拼接当前的页面url
    request_url = base_url + str(i+1) + '.shtml'
    #得到soup解析
    soup = get_page_content(request_url)
    #通过soup解析得到当前页面的DataFrame
    df = analysis(soup)
    result = result.append(df)
print(result)
result.to_excel('car_complain.xlsx', index = False)

            
    

        id    brand  car_model                           type  \
0   571157    长安马自达  马自达3 昂克赛拉       2017款 1.5L 三厢 自动 豪华型 国VI   
1   571154     一汽海马        福美来           2015款 1.6L 手动 挚爱版 M5   
2   571152    一汽-大众         宝来              2019款 1.5L 自动 舒适型   
3   571151     东风本田     本田CR-V  2019款 240TURBO CVT 两驱 风尚版 国VI   
4   571147    一汽-大众         捷达               2010款 1.6 手动 前卫型   
..     ...      ...        ...                            ...   
25  570474     长安福特         锐界   2020款 EcoBoost 245 两驱 豪锐型 7座   
26  570473     长安福特         翼搏               2013款 1.5 自动 尊贵型   
27  570468     东风本田     本田CR-V         2021款 锐·混动 2.0L 两驱 净行版   
28  570464    特斯拉中国    Model Y                2021款 长续航 全轮驱动版   
29  570463  上汽大众斯柯达       柯迪亚克       2020款 TSI330 两驱 豪华优享版 5座   

                       desc     problem    datetime status  
0         长安马自达昂克赛拉中控屏幕内屏开裂       H168,  2021-02-08   信息审核  
1       一汽海马福美来发动机异响 抖动不予解决     A9,A24,  2021-02-08   信息审核  
2         一汽大众宝来转向加油时左前车轮异响       G1